In [1]:
import os


DATASET_DIR = "../datasets"
IMAGE_DIR_TRAIN = os.path.join(DATASET_DIR, "image", "train")
MASK_DIR_TRAIN = os.path.join(DATASET_DIR, "mask", "train")
IMAGE_DIR_VAL = os.path.join(DATASET_DIR, "image", "validation")
MASK_DIR_VAL = os.path.join(DATASET_DIR, "mask", "validation")
#SUBINDO UM NIVEL NOS CAMINHOOS

In [2]:
# Bloco 2: Definição do Modelo U-Net++

import torch
import torch.nn as nn
import torch.nn.functional as F

# Bloco de Convolução usado no Encoder e Decoder
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

# Arquitetura U-Net++
class UNetPlusPlus(nn.Module):
    def __init__(self, num_classes=1, input_channels=3, deep_supervision=False):
        super(UNetPlusPlus, self).__init__()
        self.deep_supervision = deep_supervision

        # Encoder
        self.conv0_0 = ConvBlock(input_channels, 64)
        self.conv1_0 = ConvBlock(64, 128)
        self.conv2_0 = ConvBlock(128, 256)
        self.conv3_0 = ConvBlock(256, 512)
        self.conv4_0 = ConvBlock(512, 1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Decoder com densa conexão
        self.conv0_1 = ConvBlock(64 + 128, 64)
        self.conv0_2 = ConvBlock(64*2 + 128, 64)
        self.conv0_3 = ConvBlock(64*3 + 128, 64)
        self.conv0_4 = ConvBlock(64*4 + 128, 64)

        self.conv1_1 = ConvBlock(128 + 256, 128)
        self.conv1_2 = ConvBlock(128*2 + 256, 128)
        self.conv1_3 = ConvBlock(128*3 + 256, 128)

        self.conv2_1 = ConvBlock(256 + 512, 256)
        self.conv2_2 = ConvBlock(256*2 + 512, 256)

        self.conv3_1 = ConvBlock(512 + 1024, 512)

        self.upsample = lambda x, target: F.interpolate(x, size=target.shape[2:], mode='bilinear', align_corners=True)

        # Saídas
        self.final_1 = nn.Conv2d(64, num_classes, kernel_size=1)
        self.final_2 = nn.Conv2d(64, num_classes, kernel_size=1)
        self.final_3 = nn.Conv2d(64, num_classes, kernel_size=1)
        self.final_4 = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x2_0 = self.conv2_0(self.pool(x1_0))
        x3_0 = self.conv3_0(self.pool(x2_0))
        x4_0 = self.conv4_0(self.pool(x3_0))

        x0_1 = self.conv0_1(torch.cat([x0_0, self.upsample(x1_0, x0_0)], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.upsample(x2_0, x1_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.upsample(x3_0, x2_0)], 1))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.upsample(x4_0, x3_0)], 1))

        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.upsample(x1_1, x0_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.upsample(x2_1, x1_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.upsample(x3_1, x2_0)], 1))

        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.upsample(x1_2, x0_0)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.upsample(x2_2, x1_0)], 1))

        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.upsample(x1_3, x0_0)], 1))

        if self.deep_supervision:
            output1 = self.final_1(x0_1)
            output2 = self.final_2(x0_2)
            output3 = self.final_3(x0_3)
            output4 = self.final_4(x0_4)
            return [output1, output2, output3, output4]
        else:
            output = self.final_4(x0_4)
            return output

# Instanciando o modelo
num_classes = 1
model = UNetPlusPlus(num_classes=num_classes)

# Imprimindo a arquitetura do modelo
print(model)


UNetPlusPlus(
  (conv0_0): ConvBlock(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (conv1_0): ConvBlock(
    (conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (conv2_0): ConvBlock(
    (conv): Sequential(
      (0): Conv2d(128, 256

In [3]:
# Bloco Extra: Configuração do DataLoader (Igual ao U-Net Normal)

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definição do dataset customizado
class ISICDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = sorted(os.listdir(image_dir))
        self.mask_list = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        if idx >= len(self.image_list):
            idx = len(self.image_list) - 1  # Garante que não acessa um índice inexistente

        img_path = os.path.join(self.image_dir, self.image_list[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_list[idx])

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if image is None or mask is None:
            print(f"⚠️ Imagem ou Máscara corrompida em: {img_path} ou {mask_path}")
            return self.__getitem__((idx + 1) % len(self.image_list))

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256))

        image = image / 255.0
        mask = (mask > 0).astype(np.float32)

        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return image, mask


# Caminho dos datasets (relativo ao notebook)
DATASET_DIR = "../datasets"
IMAGE_DIR_TRAIN = os.path.join(DATASET_DIR, "image", "train")
MASK_DIR_TRAIN = os.path.join(DATASET_DIR, "mask", "train")
IMAGE_DIR_VAL = os.path.join(DATASET_DIR, "image", "validation")
MASK_DIR_VAL = os.path.join(DATASET_DIR, "mask", "validation")

# Criar dataset e DataLoader
train_dataset = ISICDataset(IMAGE_DIR_TRAIN, MASK_DIR_TRAIN)
val_dataset = ISICDataset(IMAGE_DIR_VAL, MASK_DIR_VAL)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, drop_last=True)

# Teste: Exibir um batch do DataLoader
batch = next(iter(train_loader))
images, masks = batch

print(f"Formato do batch de imagens: {images.shape}")  # Esperado: [8, 3, 256, 256]
print(f"Formato do batch de máscaras: {masks.shape}")  # Esperado: [8, 1, 256, 256]


Formato do batch de imagens: torch.Size([8, 3, 256, 256])
Formato do batch de máscaras: torch.Size([8, 1, 256, 256])


In [4]:
# Bloco 4: Funções de Perda e Métricas (Igual ao U-Net Normal)

import torch
import torch.nn.functional as F
import torch.optim as optim
import time

# Configuração do dispositivo (usando CPU)
device = torch.device("cpu")
model.to(device)

# Função de perda (Binary Cross Entropy com Dice Loss)
def dice_loss(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)  # Converte logits para probabilidade
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

criterion = lambda pred, target: 0.5 * F.binary_cross_entropy_with_logits(pred, target) + 0.5 * dice_loss(pred, target)

# Otimizador
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Funções para métricas
def iou(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-5) / (union + 1e-5)

def dice_coefficient(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    return (2. * intersection + 1e-5) / (pred.sum() + target.sum() + 1e-5)

def precision(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    predicted_positive = pred.sum()
    return (true_positive + 1e-5) / (predicted_positive + 1e-5)

def recall(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    actual_positive = target.sum()
    return (true_positive + 1e-5) / (actual_positive + 1e-5)

def f1_score(pred, target, threshold=0.5):
    p = precision(pred, target, threshold)
    r = recall(pred, target, threshold)
    return 2 * (p * r) / (p + r + 1e-5)


In [5]:
# Bloco 5: Função de Treinamento e Validação (Igual ao U-Net Normal)

import torch
import time

# Configuração do dispositivo (usando CPU)
device = torch.device("cpu")
model.to(device)

# Função para treinar o modelo com métricas
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        start_time = time.time()

        # Métricas de treino
        train_iou, train_dice, train_precision, train_recall, train_f1 = 0, 0, 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Calcular métricas para o batch
            train_iou += iou(outputs, masks).item()
            train_dice += dice_coefficient(outputs, masks).item()
            train_precision += precision(outputs, masks).item()
            train_recall += recall(outputs, masks).item()
            train_f1 += f1_score(outputs, masks).item()

        # Validação
        model.eval()
        val_loss = 0
        val_iou, val_dice, val_precision, val_recall, val_f1 = 0, 0, 0, 0, 0

        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, masks).item()

                # Calcular métricas para o batch
                val_iou += iou(outputs, masks).item()
                val_dice += dice_coefficient(outputs, masks).item()
                val_precision += precision(outputs, masks).item()
                val_recall += recall(outputs, masks).item()
                val_f1 += f1_score(outputs, masks).item()

        # Exibir progresso
        print(f"📌 Época [{epoch+1}/{epochs}] - Loss Treino: {epoch_loss/len(train_loader):.4f} | Loss Validação: {val_loss/len(val_loader):.4f}")
        print(f"   Métricas de Treino -> IoU: {train_iou/len(train_loader):.4f} | Dice: {train_dice/len(train_loader):.4f} | Prec: {train_precision/len(train_loader):.4f} | Rec: {train_recall/len(train_loader):.4f} | F1: {train_f1/len(train_loader):.4f}")
        print(f"   Métricas de Validação -> IoU: {val_iou/len(val_loader):.4f} | Dice: {val_dice/len(val_loader):.4f} | Prec: {val_precision/len(val_loader):.4f} | Rec: {val_recall/len(val_loader):.4f} | F1: {val_f1/len(val_loader):.4f}")
        print(f"   Tempo da Época: {time.time() - start_time:.2f}s")

# Treinar o modelo com métricas
train_model(model, train_loader, val_loader, epochs=100)


KeyboardInterrupt: 